# Import modules

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from pysheds.grid import Grid
import seaborn as sns
import rasterio
from rasterio.transform import from_bounds


# Instatiate a grid from a DEM raster

Data from MERIT DEM: Multi-Error-Removed Improved-Terrain DEM

In [ ]:
folder = 'data/input/'
out_folder = 'data/output/'

dem_file ='merit_tenerife_4326.tif'

# Load the DEM data
grid = Grid.from_raster(folder + dem_file, data_name='dem')
dem = grid.read_raster(folder + dem_file)

np.max(np.max(dem))
dem[dem<0]=0
dem[dem>6000]=0

In [ ]:
fig, ax = plt.subplots(figsize=(8,6))
fig.patch.set_alpha(0)

plt.imshow(dem, extent=grid.extent, cmap='terrain', zorder=1)
plt.colorbar(label='Elevation (m)')
plt.grid(zorder=0)
plt.title('Digital elevation map', size=14)
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.tight_layout()

# Resolve flats in DEM

In [ ]:
# Condition DEM
# ----------------------
# Fill pits in DEM
pit_filled_dem = grid.fill_pits(dem)

# Fill depressions in DEM
flooded_dem = grid.fill_depressions(pit_filled_dem)
    
# Resolve flats in DEM
inflated_dem = grid.resolve_flats(flooded_dem)

# Convert DEM to flow direction grid

In [ ]:
# Compute flow directions - D8 flow directions
# -------------------------------------
fdir = grid.flowdir(inflated_dem)

# Examine grid

In [ ]:
fdir

In [ ]:
fdir.size

# Flow directions

In [ ]:
fig = plt.figure(figsize=(8,6))
fig.patch.set_alpha(0)
plt.imshow(fdir, extent=grid.extent, cmap='viridis', zorder=2)
plt.colorbar(label='D8 direction value')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('D8 flow direction grid', size=14)
plt.grid(zorder=-1)
plt.tight_layout()

# Flow accumulation

In [ ]:
# Calculate flow accumulation
# --------------------------
acc = grid.accumulation(fdir)

In [ ]:
fig, ax = plt.subplots(figsize=(8,6))
fig.patch.set_alpha(0)
plt.grid('on', zorder=0)
im = ax.imshow(acc, extent=grid.extent, zorder=2,
               cmap='cubehelix',
               norm=colors.LogNorm(1, acc.max()),
               interpolation='bilinear')
plt.colorbar(im, ax=ax, label='Upstream Cells')
plt.title('Flow Accumulation', size=14)
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.tight_layout()

# Save flow accumulation as GeoTIFF

In [ ]:
# Define a flow accumulation threshold for stream network extraction
threshold = 1000  # Number of cells contributing to flow
streams = (acc > threshold)*1.0  # Binary mask for stream network

In [ ]:
# Save as GeoTIFF
output_tif = out_folder + 'streams_raster.tif'

# Get the affine transformation from PySheds
transform = grid.affine
nrows, ncols = streams.shape

# Save the stream network as a GeoTIFF using rasterio
with rasterio.open(
    output_tif,
    'w',
    driver='GTiff',
    height=nrows,
    width=ncols,
    count=1,  # Single band
    dtype=streams.dtype,
    crs='EPSG:4326',  # WGS84 CRS
    transform=transform,
) as dst:
    dst.write(streams,1)  # Save

# Delineate catchment

In [ ]:
#set pour point
lon = -16.801269
lat = 28.157129


# Delineate a catchment
# ---------------------
# Specify pour point
x, y = lon, lat

# Snap pour point to high accumulation cell
x_snap, y_snap = grid.snap_to_mask(acc > 100, (x, y))

# Delineate the catchment
catch = grid.catchment(x=x_snap, y=y_snap, fdir=fdir, xytype='coordinate')

In [ ]:
# Clip the bounding box to the catchment
grid.clip_to(catch)

In [ ]:
# Get a view of the catchment
clipped_catch = grid.view(catch)

In [ ]:
# Plot the catchment
fig, ax = plt.subplots(figsize=(8,6))
fig.patch.set_alpha(0)

plt.grid('on', zorder=0)
im = ax.imshow(np.where(clipped_catch, clipped_catch, np.nan), extent=grid.extent,
               zorder=1, cmap='Greys_r')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Delineated Catchment', size=14)

# Get River Network

In [ ]:
branches = grid.extract_river_network(fdir, acc > 100)

In [ ]:
sns.set_palette('husl')
fig, ax = plt.subplots(figsize=(8.5,6.5))

plt.xlim(grid.bbox[0], grid.bbox[2])
plt.ylim(grid.bbox[1], grid.bbox[3])
ax.set_aspect('equal')

for branch in branches['features']:
    line = np.asarray(branch['geometry']['coordinates'])
    plt.plot(line[:, 0], line[:, 1])
    
_ = plt.title('D8 channels', size=14)

# Get distances to upstream cells

In [ ]:
# Calculate distance to outlet from each cell
# -------------------------------------------
dist = grid.distance_to_outlet(x=x_snap, y=y_snap, fdir=fdir, xytype='coordinate')

In [ ]:
fig, ax = plt.subplots(figsize=(8,6))
fig.patch.set_alpha(0)
plt.grid('on', zorder=0)
im = ax.imshow(dist, extent=grid.extent, zorder=2,
               cmap='cubehelix_r')
plt.colorbar(im, ax=ax, label='Distance to outlet (cells)')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Flow Distance', size=14)